# Wide Residual Network

Definition of WideResNet and Architecture:

Residual Networks vs Wide Residual Networks

##  <center>The Result of Wide Residual Network </center>

* Data Preprocessing
* Model Cross Validation Results
* Evaluate the GridSearchCV Results
* Model Training and Learning Curves
* Model Adversarial Training Approach.


#### import the libraries.

In [2]:
import tensorflow as tf
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import Callback, LearningRateScheduler, EarlyStopping
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Add, Activation, Dropout, Flatten, Dense
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import cv2
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from wresnet import WideResidualNetwork

import gzip
import pickle

import numpy as np

import warnings
warnings.filterwarnings("ignore")

print("\nTensorflow Version: " + tf.__version__)


ModuleNotFoundError: ignored

<b> Data Preprocessing </b>


* Read Data from File

In [28]:
def read_data():
    with open("data.pz", 'rb') as file_:
        with gzip.GzipFile(fileobj=file_) as gzf:
            data = pickle.load(gzf, encoding='latin1', fix_imports=True)
    return data
data = read_data()

* Call data preprocessing function

In [31]:
def preprocessing_data(data):
    data_X = []
    Y_data = []
    for row in data:
        data_X.append(cv2.resize(row['crop'], (32,32)))
        Y_data.append(row['label'])
    data_X = np.array(data_X)
    X = transform_X(data_X.astype('float32'))
    Y = transform_Y(Y_data)
    return X, Y

def transform_Y(Y):
    labelencoder = LabelEncoder()
    y_df = pd.DataFrame(Y, columns=['Label'])
    y_df['Encoded'] = labelencoder.fit_transform(y_df['Label'])
    y_cat = to_categorical(y_df['Encoded'])
    return y_cat

def transform_X(X):
    img_rows, img_cols = X[0].shape

    # transform data set
    if K.image_data_format() == 'channels_first':
        X = X.reshape(X.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        X = X.reshape(X.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)
    return X
    

In [32]:
X, y = preprocessing_data(data)
X_train, X_test, Y_train, y_test = train_test_split(X, y, test_size = 0.1)
x_train, X_val, y_train, y_val = train_test_split(X_train, Y_train, test_size = 0.1)

**Utilize Functions**

* Flipping the image using data augmentation technique

In [21]:
generator = tensorflow.keras.preprocessing.image.ImageDataGenerator(rotation_range=10,
                               width_shift_range=5./32,
                               height_shift_range=5./32,)

* Print the results of the experiments

In [ ]:
def print_test(model,X_adv, X_test, y_test, epsilon):
  loss, acc = model.evaluate(X_adv,y_test)
  print("epsilon: {} and test evaluation : {}, {}".format(epsilon,loss, acc))
  SNR = 20*np.log10(np.linalg.norm(X_test)/np.linalg.norm(X_test-X_adv))
  print("SNR: {}".format(SNR))
  return loss, acc

**General Visulazation Methods**

* Show accurancy and loss function of the results

In [22]:
import matplotlib.pyplot as plt

def show_graph(hist):
  history = hist
  print(history.history.keys())
  # summarize history for accuracy
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()


## 1.) Baseline of the Model

* Explain the Baseline Model here

Give a shape regarding model base layers

### 1.1. ) Find Model Parameters using Cross Validation and GridSearch 

In [10]:
data_list = []
for  i in range(0,11):
  if i == 5: continue
  data_result  = pd.read_csv("grid_16_{0}.csv".format(str(i)), sep=";")
  data_list.append(data_result)
result = pd.concat(data_list)

#### Evaluate the Results

In [13]:
result["loss_mean"] = (result["loss1"]+result["loss2"]+result["loss3"])/3

In [14]:
result["acc_mean"] = (result["acc1"]+result["acc2"]+result["acc3"])/3

In [15]:
result['epoch'] = result['epoch_stopped1']
result['weight_decay'] = result['reg_penalty']
column_list = ["momentum", "learning rate", "epoch","batch size","weight_decay","loss_mean", "acc_mean"]

In [18]:
result.sort_values(axis=0, by="acc_mean", ascending=False)[column_list].head(3)

,momentum,learning rate,epoch,batch size,weight_decay,loss_mean,acc_mean
0,0.9,0.10,50.0,128.0,0.0001,1.096489,0.726003
3,0.9,0.10,100.0,128.0,0.0010,1.199277,0.722513
0,0.9,0.01,50.0,64.0,0.0010,1.687745,0.718441


In [20]:
result.sort_values(axis=0, by="loss_mean", ascending=True)[column_list].head(3)

,momentum,learning rate,epoch,batch size,weight_decay,loss_mean,acc_mean
2,0.9,0.01,50.0,64.0,0.0100,1.003848,0.708551
3,0.9,0.10,50.0,64.0,0.0010,1.033697,0.699825
2,0.9,0.01,50.0,128.0,0.0001,1.043016,0.689936


Results and comments here

### 1.2.) Train Baseline Model

In [27]:
wresnet_ins = WideResidualNetwork()
EPOCHS = 50
BS = 64
init = (32, 32,1)
sgd = SGD(lr=0.01, momentum=0.9)
#wrn_16_2.summary()
wrn_16_2 = wresnet_ins.create_wide_residual_network(0.0005, 0.01,init,0.9, nb_classes=4, N=2, k=2, dropout=0.0)
wrn_16_2.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["acc"])
print("Finished compiling")


conv2:channel:  -1
Wide Residual Network-16-2 created.
Finished compiling


In [1]:

hist = wrn_16_2.fit(generator.flow(x_train, y_train, batch_size=BS), steps_per_epoch=len(x_train) // BS, epochs=EPOCHS,
                   validation_data=(X_val, y_val),
                   validation_steps=X_val.shape[0] // BS,)
wrn_16_2.save("wrn_model.h5")

NameError: ignored

<b>Evaluate the Learning Curve</b>

### 1.3.) Adversarial Training on Baseline Model

### Evaluate the Results of Adversarial Traing and Non-Adversarial Training 

## 2. Complex Model

### 2.1.) Find Model Parameters using Cross Validation and GridSearch

### 2.2.) Train Model

### 2.3.) Adversarial Training

#### <center> Evaluate the Results of Adversarial Traing and Non-Adversarial Training for the Model</center>